# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Load variables
import os
from dotenv import load_dotenv
load_dotenv()

# Snowpark Imports
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

# Connect to Snowflake

In [2]:
snowflake_connection_cfg = {
    "ACCOUNT": os.getenv('SF_ACCOUNT'),
    "USER": os.getenv('SF_USER'),
    "ROLE": os.getenv('SF_ROLE'),
    "PASSWORD": os.getenv('SF_PASSWORD'),
    "DATABASE": os.getenv('SF_DATABASE'),
    "SCHEMA": os.getenv('SF_SCHEMA'),
    "WAREHOUSE": os.getenv('SF_WAREHOUSE')
}

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

print('Role:     ', session.get_current_role())
print('Warehouse:', session.get_current_warehouse())
print('Database: ', session.get_current_database())
print('Schema:   ', session.get_current_schema())

Role:      "ACCOUNTADMIN"
Warehouse: "COMPUTE_WH"
Database:  "MACHINE_LEARNING"
Schema:    "PUBLIC"


In [22]:
# Create some test data to work with
technology_texts_german = [
    "Die Blockchain-Technologie bietet eine sichere Methode zur Aufzeichnung von Transaktionen. Sie basiert auf einer dezentralen Datenbank, die es ermöglicht, Informationen über Netzwerkknoten hinweg zu verteilen und zu verifizieren. Diese Technologie wird häufig in der Finanzwelt verwendet, insbesondere für Kryptowährungen wie Bitcoin.",
    "Quantencomputer nutzen die Prinzipien der Quantenmechanik, um Informationen zu verarbeiten. Im Gegensatz zu klassischen Computern, die Daten in Bits verarbeiten, verwenden Quantencomputer Qubits, die gleichzeitig mehrere Zustände einnehmen können. Dies ermöglicht es ihnen, komplexe Berechnungen schneller durchzuführen, was sie besonders nützlich für Aufgaben wie die Materialwissenschaft oder die Kryptographie macht.",
    "Virtuelle Realität (VR) schafft eine künstlich generierte Welt mit Hilfe von Computergrafiken. Benutzer von VR-Brillen können in diese Welten eintauchen und interagieren, was das Gefühl vermittelt, physisch an einem anderen Ort zu sein. VR wird zunehmend in Bereichen wie Bildung, Training, Unterhaltung und sogar in der Medizin für therapeutische Zwecke eingesetzt."
]

df = session.create_dataframe(technology_texts_german, schema=['"context"'])
df.show(n=15, max_width=1000)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"context"                                                                                                                                                                                                                                                                                                                                                                                                                            |
--------------------------------------------------------------------------------------------------------------------------------------------------------

# Register & Run Question-Answer Model

In [18]:
# Get the model registry object
from snowflake.ml.registry import Registry
reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
    )

In [31]:
# Get the embedding model from Huggingface
# Make sure it fits into a Snowflake warehouse and does not require GPUs
# Otherwise the model must deployed in Snowpark Container Services
from transformers import pipeline
pipe = pipeline("question-answering", model="deepset/gelectra-base-germanquad")

# Register the model to Snowflake (predict is the model's function we want to call)
snow_model_custom = reg.log_model(
    pipe, 
    model_name='gelectra_base_germanquad', 
    sample_input_data=df.limit(10),
    conda_dependencies=['tokenizers','transformers','sentencepiece']
    )

# Model Output
answers = snow_model_custom.run(df.select(F.lit('Wofür wird Blockchain hauptsächlich genutzt?').alias('"question"'), F.col('"context"'))).cache_result()
answers.select(['"context"','"score"','"answer"']).show(max_width=100)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"context"                                                                                             |"score"              |"answer"                                                                            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Die Blockchain-Technologie bietet eine sichere Methode zur Aufzeichnung von Transaktionen. Sie ba...  |0.29910144209861755  |in der Finanzwelt                                                                   |
|Quantencomputer nutzen die Prinzipien der Quantenmechanik, um Informationen zu verarbeiten. Im Ge...  |0.14709903299808502  |Aufgaben wie die Materialw